<a href="https://colab.research.google.com/github/jyellow/ai-repo/blob/main/thirdpart/Dive-into-DL-TensorFlow2.0-master/code/chapter05_CNN/5.6_alexnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 深度卷积神经网络（AlexNet）

在LeNet提出后的将近20年里，神经网络一度被其他机器学习方法超越，如支持向量机。虽然LeNet可以在早期的小数据集上取得好的成绩，但是在更大的真实数据集上的表现并不尽如人意。一方面，神经网络计算复杂。虽然20世纪90年代也有过一些针对神经网络的加速硬件，但并没有像之后GPU那样大量普及。因此，训练一个多通道、多层和有大量参数的卷积神经网络在当年很难完成。另一方面，当年研究者还没有大量深入研究参数初始化和非凸优化算法等诸多领域，导致复杂的神经网络的训练通常较困难。

我们在上一节看到，神经网络可以直接基于图像的原始像素进行分类。这种称为端到端（end-to-end）的方法节省了很多中间步骤。然而，在很长一段时间里更流行的是研究者通过勤劳与智慧所设计并生成的手工特征。这类图像分类研究的主要流程是：

1. 获取图像数据集；
2. 使用已有的特征提取函数生成图像的特征；
3. 使用机器学习模型对图像的特征分类。

当时认为的机器学习部分仅限最后这一步。如果那时候跟机器学习研究者交谈，他们会认为机器学习既重要又优美。优雅的定理证明了许多分类器的性质。机器学习领域生机勃勃、严谨而且极其有用。然而，如果跟计算机视觉研究者交谈，则是另外一幅景象。他们会告诉你图像识别里“不可告人”的现实是：计算机视觉流程中真正重要的是数据和特征。也就是说，使用较干净的数据集和较有效的特征甚至比机器学习模型的选择对图像分类结果的影响更大。


## 学习特征表示

既然特征如此重要，它该如何表示呢？

我们已经提到，在相当长的时间里，特征都是基于各式各样手工设计的函数从数据中提取的。事实上，不少研究者通过提出新的特征提取函数不断改进图像分类结果。这一度为计算机视觉的发展做出了重要贡献。

然而，另一些研究者则持异议。他们认为特征本身也应该由学习得来。他们还相信，为了表征足够复杂的输入，特征本身应该分级表示。持这一想法的研究者相信，多层神经网络可能可以学得数据的多级表征，并逐级表示越来越抽象的概念或模式。以图像分类为例，并回忆[“二维卷积层”](conv-layer.ipynb)一节中物体边缘检测的例子。在多层神经网络中，图像的第一级的表示可以是在特定的位置和⻆度是否出现边缘；而第二级的表示说不定能够将这些边缘组合出有趣的模式，如花纹；在第三级的表示中，也许上一级的花纹能进一步汇合成对应物体特定部位的模式。这样逐级表示下去，最终，模型能够较容易根据最后一级的表示完成分类任务。需要强调的是，输入的逐级表示由多层模型中的参数决定，而这些参数都是学出来的。

尽管一直有一群执着的研究者不断钻研，试图学习视觉数据的逐级表征，然而很长一段时间里这些野心都未能实现。这其中有诸多因素值得我们一一分析。


### 缺失要素一：数据

包含许多特征的深度模型需要大量的有标签的数据才能表现得比其他经典方法更好。限于早期计算机有限的存储和90年代有限的研究预算，大部分研究只基于小的公开数据集。例如，不少研究论文基于加州大学欧文分校（UCI）提供的若干个公开数据集，其中许多数据集只有几百至几千张图像。这一状况在2010年前后兴起的大数据浪潮中得到改善。特别是，2009年诞生的ImageNet数据集包含了1,000大类物体，每类有多达数千张不同的图像。这一规模是当时其他公开数据集无法与之相提并论的。ImageNet数据集同时推动计算机视觉和机器学习研究进入新的阶段，使此前的传统方法不再有优势。


### 缺失要素二：硬件

深度学习对计算资源要求很高。早期的硬件计算能力有限，这使训练较复杂的神经网络变得很困难。然而，通用GPU的到来改变了这一格局。很久以来，GPU都是为图像处理和计算机游戏设计的，尤其是针对大吞吐量的矩阵和向量乘法从而服务于基本的图形变换。值得庆幸的是，这其中的数学表达与深度网络中的卷积层的表达类似。通用GPU这个概念在2001年开始兴起，涌现出诸如OpenCL和CUDA之类的编程框架。这使得GPU也在2010年前后开始被机器学习社区使用。


## AlexNet

2012年，AlexNet横空出世。这个模型的名字来源于论文第一作者的姓名Alex Krizhevsky [1]。AlexNet使用了8层卷积神经网络，并以很大的优势赢得了ImageNet 2012图像识别挑战赛。它首次证明了学习到的特征可以超越手工设计的特征，从而一举打破计算机视觉研究的前状。

AlexNet与LeNet的设计理念非常相似，但也有显著的区别。

第一，与相对较小的LeNet相比，AlexNet包含8层变换，其中有5层卷积和2层全连接隐藏层，以及1个全连接输出层。下面我们来详细描述这些层的设计。

AlexNet第一层中的卷积窗口形状是$11\times11$。因为ImageNet中绝大多数图像的高和宽均比MNIST图像的高和宽大10倍以上，ImageNet图像的物体占用更多的像素，所以需要更大的卷积窗口来捕获物体。第二层中的卷积窗口形状减小到$5\times5$，之后全采用$3\times3$。此外，第一、第二和第五个卷积层之后都使用了窗口形状为$3\times3$、步幅为2的最大池化层。而且，AlexNet使用的卷积通道数也大于LeNet中的卷积通道数数十倍。

紧接着最后一个卷积层的是两个输出个数为4096的全连接层。这两个巨大的全连接层带来将近1 GB的模型参数。由于早期显存的限制，最早的AlexNet使用双数据流的设计使一个GPU只需要处理一半模型。幸运的是，显存在过去几年得到了长足的发展，因此通常我们不再需要这样的特别设计了。

第二，AlexNet将sigmoid激活函数改成了更加简单的ReLU激活函数。一方面，ReLU激活函数的计算更简单，例如它并没有sigmoid激活函数中的求幂运算。另一方面，ReLU激活函数在不同的参数初始化方法下使模型更容易训练。这是由于当sigmoid激活函数输出极接近0或1时，这些区域的梯度几乎为0，从而造成反向传播无法继续更新部分模型参数；而ReLU激活函数在正区间的梯度恒为1。因此，若模型参数初始化不当，sigmoid函数可能在正区间得到几乎为0的梯度，从而令模型无法得到有效训练。

第三，AlexNet通过丢弃法（参见[“丢弃法”](../chapter_deep-learning-basics/dropout.ipynb)一节）来控制全连接层的模型复杂度。而LeNet并没有使用丢弃法。

第四，AlexNet引入了大量的图像增广，如翻转、裁剪和颜色变化，从而进一步扩大数据集来缓解过拟合。我们将在后面的[“图像增广”](../chapter_computer-vision/image-augmentation.ipynb)一节详细介绍这种方法。

下面我们实现稍微简化过的AlexNet。

In [1]:
import tensorflow as tf
print(tf.__version__)

2.17.0


In [2]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

# Your output is probably something like ['/device:CPU:0']
# It should be ['/device:CPU:0', '/device:GPU:0']

['/device:CPU:0', '/device:GPU:0']


建议采用GPU进行训练，需要使用tensorflow-gpu-2.0并设置memory_growth:

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
        print("当前可用的GPU设备:", gpu)  # 输出当前的GPU设备
else:
    print("没有检测到可用的GPU设备。")

当前可用的GPU设备: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


### 整体架构
这是一个包含14层的深度卷积神经网络，主要分为以下几个部分：

### 卷积部分（1-8层）
1. **初始特征提取**
   - 第1层：使用96个大尺寸(11×11)的卷积核进行初步特征提取，步长为4
   - 第2层：最大池化层压缩特征图尺寸

2. **中层特征提取**
   - 第3层：使用256个中等尺寸(5×5)的卷积核提取更复杂的特征
   - 第4层：再次进行最大池化

3. **精细特征提取**
   - 第5-7层：使用多个小尺寸(3×3)的卷积层，逐步提取更高级的特征
   - 第8层：最后一次池化操作

### 全连接部分（9-14层）
1. **特征整合**
   - 第9层：展平层，将卷积特征转为一维向量

2. **分类器**
   - 第10-12层：两个具有4096个神经元的大型全连接层
   - 第11、13层：使用Dropout(0.5)防止过拟合
   - 第14层：输出层，10个神经元对应10个类别

### 特点
- 使用ReLU激活函数提高训练效率
- 采用多层卷积逐步提取特征
- 使用Dropout防止过拟合
- 最后使用sigmoid激活函数输出分类概率

这个网络结构在当时(2012年)是突破性的，它标志着深度学习在计算机视觉领域的重要里程碑。

### 卷积层参数计算
1. **第一卷积层**
- 参数 = (11×11×1×96) + 96 = 11,616
  - 卷积核：11×11×1(输入通道)×96(过滤器数量)
  - 偏置：96

2. **第三卷积层**
- 参数 = (5×5×96×256) + 256 = 614,656
  - 卷积核：5×5×96(输入通道)×256(过滤器数量)
  - 偏置：256

3. **第五卷积层**
- 参数 = (3×3×256×384) + 384 = 885,120
  - 卷积核：3×3×256(输入通道)×384(过滤器数量)
  - 偏置：384

4. **第六卷积层**
- 参数 = (3×3×384×384) + 384 = 1,327,488
  - 卷积核：3×3×384(输入通道)×384(过滤器数量)
  - 偏置：384

5. **第七卷积层**
- 参数 = (3×3×384×256) + 256 = 884,992
  - 卷积核：3×3×384(输入通道)×256(过滤器数量)
  - 偏置：256

### 全连接层参数计算
6. **第一全连接层**
- 参数 = (上一层输出×4096) + 4096
  - 具体数值取决于前面卷积层的输出尺寸

7. **第二全连接层**
- 参数 = (4096×4096) + 4096 = 16,781,312
  - 权重：4096×4096
  - 偏置：4096

8. **输出层**
- 参数 = (4096×10) + 10 = 40,970
  - 权重：4096×10
  - 偏置：10

### 注意事项
- 池化层和Dropout层没有可训练参数
- 实际参数总数会非常大，这也是为什么AlexNet在当时需要使用双GPU训练
- 最大的参数量集中在全连接层，这也是后来的网络架构(如VGG、ResNet等)试图改进的地方

In [4]:
net = tf.keras.models.Sequential([
    # 第一层：卷积层，96个11x11的卷积核，步幅为4，激活函数为ReLU
    tf.keras.layers.Conv2D(filters=96, kernel_size=11, strides=4, activation='relu'),

    # 第二层：最大池化层，池化窗口为3x3，步幅为2
    tf.keras.layers.MaxPool2D(pool_size=3, strides=2),

    # 第三层：卷积层，256个5x5的卷积核，填充方式为'same'，激活函数为ReLU
    tf.keras.layers.Conv2D(filters=256, kernel_size=5, padding='same', activation='relu'),

    # 第四层：最大池化层，池化窗口为3x3，步幅为2
    tf.keras.layers.MaxPool2D(pool_size=3, strides=2),

    # 第五层：卷积层，384个3x3的卷积核，填充方式为'same'，激活函数为ReLU
    tf.keras.layers.Conv2D(filters=384, kernel_size=3, padding='same', activation='relu'),

    # 第六层：卷积层，384个3x3的卷积核，填充方式为'same'，激活函数为ReLU
    tf.keras.layers.Conv2D(filters=384, kernel_size=3, padding='same', activation='relu'),

    # 第七层：卷积层，256个3x3的卷积核，填充方式为'same'，激活函数为ReLU
    tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'),

    # 第八层：最大池化层，池化窗口为3x3，步幅为2
    tf.keras.layers.MaxPool2D(pool_size=3, strides=2),

    # 第九层：展平层，将多维输入一维化
    tf.keras.layers.Flatten(),

    # 第十层：全连接层，4096个神经元，激活函数为ReLU
    tf.keras.layers.Dense(4096, activation='relu'),

    # 第十一层：丢弃层，丢弃率为0.5，用于防止过拟合
    tf.keras.layers.Dropout(0.5),

    # 第十二层：全连接层，4096个神经元，激活函数为ReLU
    tf.keras.layers.Dense(4096, activation='relu'),

    # 第十三层：丢弃层，丢弃率为0.5，用于防止过拟合
    tf.keras.layers.Dropout(0.5),

    # 第十四层：输出层，10个神经元，激活函数为sigmoid
    tf.keras.layers.Dense(10, activation='sigmoid')
])

我们构造一个高和宽均为224的单通道数据样本来观察每一层的输出形状。



让我根据输出结果详细分析每一层的结构和参数数量：

### 1. 卷积层部分

1. **第一卷积层 (conv2d)**
- 输入: (1, 224, 224, 1)
- 输出: (1, 54, 54, 96)
- 参数: (11×11×1×96) + 96 = 11,616

2. **第一池化层 (max_pooling2d)**
- 输入: (1, 54, 54, 96)
- 输出: (1, 26, 26, 96)
- 参数: 0 (池化层无参数)

3. **第二卷积层 (conv2d_1)**
- 输入: (1, 26, 26, 96)
- 输出: (1, 26, 26, 256)
- 参数: (5×5×96×256) + 256 = 614,656

4. **第二池化层 (max_pooling2d_1)**
- 输入: (1, 26, 26, 256)
- 输出: (1, 12, 12, 256)
- 参数: 0

5. **第三卷积层 (conv2d_2)**
- 输入: (1, 12, 12, 256)
- 输出: (1, 12, 12, 384)
- 参数: (3×3×256×384) + 384 = 885,120

6. **第四卷积层 (conv2d_3)**
- 输入: (1, 12, 12, 384)
- 输出: (1, 12, 12, 384)
- 参数: (3×3×384×384) + 384 = 1,327,488

7. **第五卷积层 (conv2d_4)**
- 输入: (1, 12, 12, 384)
- 输出: (1, 12, 12, 256)
- 参数: (3×3×384×256) + 256 = 884,992

8. **第三池化层 (max_pooling2d_2)**
- 输入: (1, 12, 12, 256)
- 输出: (1, 5, 5, 256)
- 参数: 0

### 2. 全连接层部分

9. **展平层 (flatten)**
- 输入: (1, 5, 5, 256)
- 输出: (1, 6400)
- 参数: 0

10. **第一全连接层 (dense)**
- 输入: (1, 6400)
- 输出: (1, 4096)
- 参数: (6400×4096) + 4096 = 26,218,496

11. **第一丢弃层 (dropout)**
- 输入/输出: (1, 4096)
- 参数: 0

12. **第二全连接层 (dense_1)**
- 输入: (1, 4096)
- 输出: (1, 4096)
- 参数: (4096×4096) + 4096 = 16,781,312

13. **第二丢弃层 (dropout_1)**
- 输入/输出: (1, 4096)
- 参数: 0

14. **输出层 (dense_2)**
- 输入: (1, 4096)
- 输出: (1, 10)
- 参数: (4096×10) + 10 = 40,970

### 总参数量
总计约46,764,650个参数，其中：
- 卷积层部分：约3.7百万参数
- 全连接层部分：约43百万参数

这个结果显示了AlexNet的一个重要特点：大部分参数都集中在全连接层，这也是后续网络架构改进的重点方向。

In [5]:
X = tf.random.uniform((1,224,224,1))
for layer in net.layers:
    X = layer(X)
    print(layer.name, 'output shape\t', X.shape)

conv2d output shape	 (1, 54, 54, 96)
max_pooling2d output shape	 (1, 26, 26, 96)
conv2d_1 output shape	 (1, 26, 26, 256)
max_pooling2d_1 output shape	 (1, 12, 12, 256)
conv2d_2 output shape	 (1, 12, 12, 384)
conv2d_3 output shape	 (1, 12, 12, 384)
conv2d_4 output shape	 (1, 12, 12, 256)
max_pooling2d_2 output shape	 (1, 5, 5, 256)
flatten output shape	 (1, 6400)
dense output shape	 (1, 4096)
dropout output shape	 (1, 4096)
dense_1 output shape	 (1, 4096)
dropout_1 output shape	 (1, 4096)
dense_2 output shape	 (1, 10)


## 读取数据

虽然论文中AlexNet使用ImageNet数据集，但因为ImageNet数据集训练时间较长，我们仍用前面的Fashion-MNIST数据集来演示AlexNet。读取数据的时候我们额外做了一步将图像高和宽扩大到AlexNet使用的图像高和宽224。这个可以通过`tf.image.resize_with_pad`来实现。



这是一个数据加载器类的实现，我用中文为您详细解释：

### DataLoader类的主要功能
这个类主要用于加载和预处理Fashion-MNIST数据集，包含以下功能：

### 1. 初始化函数 (`__init__`)
- 加载Fashion-MNIST数据集，获取训练集和测试集
- 对图像数据进行预处理：
  - 将像素值转换为浮点数并归一化到[0,1]区间
  - 增加通道维度(使单通道图像符合CNN输入要求)
- 将标签转换为整数类型
- 记录训练集和测试集的样本数量

### 2. 训练数据批次获取 (`get_batch_train`)
- 从训练集中随机抽取指定大小的批次
- 将图像尺寸调整到224×224(适配AlexNet的输入要求)
- 返回处理后的图像数据和对应标签

### 3. 测试数据批次获取 (`get_batch_test`)
- 功能与训练数据获取类似
- 从测试集中随机抽取数据
- 同样将图像调整到224×224大小

### 使用示例
代码最后展示了如何使用这个数据加载器：
- 设置批次大小为128
- 创建数据加载器实例
- 获取一个训练数据批次
- 打印数据形状以验证

这个数据加载器的设计体现了深度学习中数据预处理的重要性，包括数据标准化、维度处理和批次划分等关键步骤。

```python
self.train_images = np.expand_dims(self.train_images.astype(np.float32) / 255.0, axis=-1)
self.test_images = np.expand_dims(self.test_images.astype(np.float32) / 255.0, axis=-1)
```

这两行代码主要在处理训练和测试图像数据，具体做了以下几件事：

1. **数据归一化**：
   - 通过 `/255.0` 将像素值从 0-255 的范围归一化到 0-1 的范围
   - 这样做可以让模型更容易训练，防止数值过大

2. **数据类型转换**：
   - `.astype(np.float32)` 将数据类型转换为32位浮点数
   - 这是深度学习框架常用的数据类型

3. **增加维度**：
   - `np.expand_dims(..., axis=-1)` 在数组的最后增加一个维度
   - 比如原来的形状是 (60000, 28, 28)，处理后变成 (60000, 28, 28, 1)
   - 这通常是为了适配卷积神经网络的输入格式，最后一维表示通道数

这些操作分别应用在训练数据(`train_images`)和测试数据(`test_images`)上，目的是准备好适合深度学习模型使用的数据格式。

In [6]:
import numpy as np

class DataLoader:
    def __init__(self):
        """
        初始化数据加载器，加载Fashion-MNIST数据集，并进行预处理。
        """
        fashion_mnist = tf.keras.datasets.fashion_mnist
        (self.train_images, self.train_labels), (self.test_images, self.test_labels) = fashion_mnist.load_data()
        # 将训练和测试图像转换为浮点数，并归一化到[0, 1]区间，增加通道维度
        self.train_images = np.expand_dims(self.train_images.astype(np.float32) / 255.0, axis=-1)
        self.test_images = np.expand_dims(self.test_images.astype(np.float32) / 255.0, axis=-1)
        # 将训练和测试标签转换为整数
        self.train_labels = self.train_labels.astype(np.int32)
        self.test_labels = self.test_labels.astype(np.int32)
        # 记录训练和测试数据的数量
        self.num_train, self.num_test = self.train_images.shape[0], self.test_images.shape[0]

    def get_batch_train(self, batch_size):
        """
        从训练数据中随机获取一个批次的数据，并将图像resize到224x224。

        Args:
            batch_size (int): 批次大小。

        Returns:
            tuple: 包含resize后的图像和对应标签的元组。
        """
        index = np.random.randint(0, np.shape(self.train_images)[0], batch_size)
        resized_images = tf.image.resize_with_pad(self.train_images[index], 224, 224)
        return resized_images.numpy(), self.train_labels[index]

    def get_batch_test(self, batch_size):
        """
        从测试数据中随机获取一个批次的数据，并将图像resize到224x224。

        Args:
            batch_size (int): 批次大小。

        Returns:
            tuple: 包含resize后的图像和对应标签的元组。
        """
        index = np.random.randint(0, np.shape(self.test_images)[0], batch_size)
        resized_images = tf.image.resize_with_pad(self.test_images[index], 224, 224)
        return resized_images.numpy(), self.test_labels[index]

# 设置批次大小
batch_size = 128
# 创建数据加载器实例
data_loader = DataLoader()
# 从训练数据中获取一个批次的数据
x_batch, y_batch = data_loader.get_batch_train(batch_size)
print("x_batch shape:", x_batch.shape, "y_batch shape:", y_batch.shape)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
x_batch shape: (128, 224, 224, 1) y_batch shape: (128,)


## 训练

这时候我们可以开始训练AlexNet了。相对于上一节的LeNet，这里的主要改动是使用了更小的学习率。

注：这里省略了训练过程的输出，如果您需要进行训练，请执行train_alexnet()函数

In [7]:
# 定义训练函数

# 定义优化器，使用随机梯度下降（SGD）算法
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01,
                                    momentum=0.0,
                                    nesterov=False)

# 编译模型
net.compile(optimizer=optimizer,  # 指定优化器
            loss='sparse_categorical_crossentropy',  # 使用稀疏分类交叉熵作为损失函数
            metrics=['accuracy'])  # 评估指标为准确率


# 获取一个批次的训练数据并进行初步训练
# 获取初始批次数据
x_batch, y_batch = data_loader.get_batch_train(batch_size)
# 使用初始批次数据训练模型
net.fit(x_batch, y_batch)


4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.1375 - loss: 2.3010


### 训练函数的主要结构
`train_alexnet()` 函数实现了完整的模型训练过程：

### 1. 训练参数设置
- 设置总训练轮数(epoch)为5轮
- 计算每轮需要的迭代次数：用训练集大小除以批次大小(batch_size)

### 2. 训练循环结构
使用两层嵌套循环进行训练：
- 外层循环：遍历每个epoch
- 内层循环：遍历每个epoch中的所有批次

### 3. 训练步骤
每次迭代中：
- 使用DataLoader获取一个批次的训练数据
- 使用`net.fit()`方法训练模型
- 每20次迭代保存一次模型权重到文件"5.6_alexnet.weights.h5"

### 4. 模型保存
- 定期保存模型权重
- 使用h5格式存储，这是深度学习模型常用的权重保存格式

### 特点
- 采用批量训练方式
- 定期保存模型权重，防止训练中断导致进度丢失
- 使用简单的训练循环结构，便于理解和修改

最后通过调用`train_alexnet()`函数开始完整的训练过程。



这是AlexNet模型的训练函数实现，我用中文为您详细解释：

### 训练函数的主要结构
`train_alexnet()` 函数实现了完整的模型训练过程：

### 1. 训练参数设置
- 设置总训练轮数(epoch)为5轮
- 计算每轮需要的迭代次数：用训练集大小除以批次大小(batch_size)

### 2. 训练循环结构
使用两层嵌套循环进行训练：
- 外层循环：遍历每个epoch
- 内层循环：遍历每个epoch中的所有批次

### 3. 训练步骤
每次迭代中：
- 使用DataLoader获取一个批次的训练数据
- 使用`net.fit()`方法训练模型
- 每20次迭代保存一次模型权重到文件"5.6_alexnet.weights.h5"

### 4. 模型保存
- 定期保存模型权重
- 使用h5格式存储，这是深度学习模型常用的权重保存格式

### 特点
- 采用批量训练方式
- 定期保存模型权重，防止训练中断导致进度丢失
- 使用简单的训练循环结构，便于理解和修改

最后通过调用`train_alexnet()`函数开始完整的训练过程。

In [ ]:
def train_alexnet():
    # 设置训练的轮数为5
    epoch = 5
    # 计算每个epoch中的迭代次数
    num_iter = data_loader.num_train // batch_size
    # 遍历每个epoch
    for e in range(epoch):
        # 遍历每个迭代
        for n in range(num_iter):
            # 获取当前批次的训练数据
            x_batch, y_batch = data_loader.get_batch_train(batch_size)
            # 使用当前批次数据训练模型
            net.fit(x_batch, y_batch)
            # 每20次迭代保存一次模型权重
            if n % 20 == 0:
                # 保存模型权重到指定文件
                net.save_weights("5.6_alexnet.weights.h5")

# 调用训练函数开始完整的训练过程
train_alexnet()

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.0667 - loss: 2.3059
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.1250 - loss: 2.3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.0979 - loss: 2.3016
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.1823 - loss: 2.2955
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.1271 - loss: 2.2989
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.1604 - loss: 2.2940
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.2000 - loss: 2.2951
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.1365 - loss: 2.2970
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.1250 - loss: 2.2953
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.2125 - loss: 2.2865
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.1052 - loss: 2.2896
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.2417 - loss: 2.2809
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.2115 - loss: 2.2821
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.1760 - loss:

我们将训练好的参数读入，然后取测试数据计算测试准确率

In [ ]:
net.load_weights("5.6_alexnet.weights.h5")

x_test, y_test = data_loader.get_batch_test(5000)
net.evaluate(x_test, y_test, verbose=2)

157/157 - 1s - loss: 0.3022 - accuracy: 0.8856 - 1s/epoch - 8ms/step


[0.3022325336933136, 0.8855999708175659]

## 小结

* AlexNet跟LeNet结构类似，但使用了更多的卷积层和更大的参数空间来拟合大规模数据集ImageNet。它是浅层神经网络和深度神经网络的分界线。
* 虽然看上去AlexNet的实现比LeNet的实现也就多了几行代码而已，但这个观念上的转变和真正优秀实验结果的产生令学术界付出了很多年。